<a href="https://colab.research.google.com/github/davidbaines/translate_docx/blob/main/MyDrive_copy_of_Text_Translation_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This Notebook shows a full pipeline for Translation using Facebook's No Language Left Behind (NLLB) model. 

Translate text from our original language to whatever language NLLB supports. 

Source: https://colab.research.google.com/drive/1fsbzykS5ANEMVcn7gtp8Wl7gkmRzbwOW

Webpage: https://medium.com/mlearning-ai/text-translation-using-nllb-and-huggingface-tutorial-7e789e0f7816

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# download the language model pretrained file
!wget https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin
!pip install fasttext

import fasttext

pretrained_lang_model = "/content/lid218e.bin" # path of pretrained model file
model = fasttext.load_model(pretrained_lang_model)

text = "صباح الخير، الجو جميل اليوم والسماء صافية."
predictions = model.predict(text, k=1) 
print(predictions)
input_lang = predictions[0][0].replace('__label__', '')
print(input_lang)

--2023-02-09 11:01:37--  https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1176355829 (1.1G) [application/octet-stream]
Saving to: ‘lid218e.bin’

lid218e.bin         100%[===================>]   1.09G  2.82MB/s    in 6m 20s  

2023-02-09 11:07:58 (2.95 MB/s) - ‘lid218e.bin’ saved [1176355829/1176355829]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4396491 sha256=b24093982226c66ef5bf90fe78bcebef702cc7ac6385defc7c43e143dab7

# Downloads and Imports

In [3]:
!pip install -U pip transformers
!pip install sentencepiece
!pip install python-docx
!pip install nltk
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=p

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import docx
import glob
from pathlib import Path

# Download NLLB model

In [5]:
# Smallest 600M parameter model - distilled
# checkpoint = 'facebook/nllb-200-distilled-600M'

# Medium 1.3B parameter model - distilled
# checkpoint = 'facebook/nllb-200-distilled-1.3B'

# Medium 1.3B parameter model
# checkpoint = 'facebook/nllb-200-1.3B'

# Large 3.3B parameter model
checkpoint = 'facebook/nllb-200-3.3B'


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Test tokenizer and translation.

In [7]:
#Test sentence tokenizer:
para = "Hello World. It's good to see you. Thanks for buying this book."
sent_tokenize(para)

['Hello World.', "It's good to see you.", 'Thanks for buying this book.']

In [8]:
input_lang = 'eng_Latn'
target_lang = 'spa_Latn'
translation_pipeline = pipeline('translation', 
                                model=model, 
                                tokenizer=tokenizer, 
                                src_lang=input_lang, 
                                tgt_lang=target_lang, 
                                max_length = 400)
output = translation_pipeline(para)
print(output[0]['translation_text'])

Hola mundo, es bueno verte, gracias por comprar este libro.


# Define Functions

In [9]:
def get_paragraphs_from_docx(file):
        
    paras = []
    # Open connection to Word Document
    doc = docx.Document(file)
    
    # read in each paragraph in file and store the style name with it.
    for para in doc.paragraphs:
        this_para = {'style': para.style.name}
        sentences = []
        for sentence in sent_tokenize(para.text):
            sentences.append(sentence)
        this_para['sentences'] = sentences

    #print(f'Found {len(styles_in_doc)} styles {styles_in_doc} in this document.')    
    return paras


def translate_docx(file):
    
    paras = []

    # Open connection to Word Document
    doc = docx.Document(file)  

    # read in each paragraph in file and store the style name with it.
    for para in doc.paragraphs:
        this_para = {'style': para.style.name}
        sentences = [sentence for sentence in sent_tokenize(para.text)]
        translations = [translation_pipeline(sentence)[0]['translation_text'] for sentence in sentences]

        this_para['sentences'] = sentences
        this_para['translations'] = translations
        paras.append(this_para)
        
        # This line was a great simplification of the find and replace code.
        para.text = " ".join(translations)

        # I'm not sure this is required, since the style shouldn't have changed.
        para.style = this_para['style']

        
        #print(this_para)

    doc.save(file)
    #print(f'Found {len(styles_in_doc)} styles {styles_in_doc} in this document.')    
    return paras

In [10]:
input_lang = 'eng_Latn'
output_lang = 'tpi_Latn'

translation_pipeline = pipeline('translation', 
                        model=model, 
                        tokenizer=tokenizer, 
                        src_lang=input_lang, 
                        tgt_lang=output_lang, 
                        max_length = 400)

In [11]:
input_folder = Path("drive/MyDrive/EIL-Mark")
output_folder = Path("drive/MyDrive/EIL-Mark-TokPisin-3.3B")
ext_in = 'docx'
ext_out = 'docx'

In [12]:
from collections import Counter
from tqdm import tqdm
pretrained_lang_model = "/content/lid218e.bin" # path of pretrained model file
fasttext_model = fasttext.load_model(pretrained_lang_model)

def get_languages(file):

    file = file.resolve()

    #print(f"Opening {file}")
    # Open the input file as a Word document    
    try :
        document = docx.Document(file)
    except BadZipFile:
        print(f"BadZipFile Error on opening {file}")
    
    paragraphs = [para for para in document.paragraphs]
    sentences = [sentence for para in document.paragraphs for sentence in sent_tokenize(para.text)]
    
    languages = Counter()
    for sentence in sentences:
        predictions = fasttext_model.predict(sentence, k=1) 
        #print(predictions)
        output_lang = predictions[0][0].replace('__label__', '')
        #print(output_lang)
        languages.update([output_lang])

    return languages


def show_languages(files):
    results = {}

    for file in tqdm(files):
        file = file.resolve()
        languages = get_languages(file)

        #print(languages)
        #print(f"There are {len(sentences)} sentences in {output_file}.")
        results[file] = {"languages": languages, "sentence_count": len(sentences)}

    for file, info in results.items():
        print(f"There are {info['sentence_count']} sentences in {str(file)[23:]}    {info['languages'].most_common()}")
        #print(f"{info['languages']}")

In [19]:
folder = Path("drive/MyDrive/EIL-Mark-TokPisin-3.3B")
files = [file for file in folder.rglob("*." + ext_in)]
print(f"Found {len(files)} {ext_in} files in {folder.resolve()}")

for i, file in enumerate(files,1):
    file = file.resolve()
    languages_in_file = get_languages(file)
    top_language_in_file = languages_in_file.most_common(1)[0][0]
    file_is_english = top_language_in_file == "eng_Latn"

    #print(top_language_in_file, file_is_english)

    if file_is_english:
        print(f"{i:>4} : Translating file {file} from English to Tok Pisin.")
        try :
            document = docx.Document(file)
        except BadZipFile:
            print(f"BadZipFile Error on opening {file}")
            continue
        
        # Save the file.
        document.save(file)

        # Translate the content
        paragraphs = translate_docx(file)
        
        print(f"{i:>4} : Translated file {file} from English to Tok Pisin")
    else:
        print(f"{i:>4} : Not translating file {file}. It seems to be in :{top_language_in_file}.")

            
    

Found 321 docx files in /content/drive/MyDrive/EIL-Mark-TokPisin-3.3B
   1 : Not translating file /content/drive/MyDrive/EIL-Mark-TokPisin-3.3B/0.intro/Mark Introduction 1.docx. It seems to be in :tpi_Latn.
   2 : Not translating file /content/drive/MyDrive/EIL-Mark-TokPisin-3.3B/10.1-10.12/5 - Filling the Gaps.docx. It seems to be in :tpi_Latn.
   3 : Not translating file /content/drive/MyDrive/EIL-Mark-TokPisin-3.3B/10.1-10.12/3 - Defining the Scenes.docx. It seems to be in :tpi_Latn.
   4 : Not translating file /content/drive/MyDrive/EIL-Mark-TokPisin-3.3B/10.1-10.12/4 - Embodying the Text.docx. It seems to be in :tpi_Latn.
   5 : Not translating file /content/drive/MyDrive/EIL-Mark-TokPisin-3.3B/10.1-10.12/1 - Hear and Heart.docx. It seems to be in :tpi_Latn.
   6 : Not translating file /content/drive/MyDrive/EIL-Mark-TokPisin-3.3B/10.1-10.12/2 - Setting the Stage.docx. It seems to be in :tpi_Latn.
   7 : Not translating file /content/drive/MyDrive/EIL-Mark-TokPisin-3.3B/10.13-10.3

In [ ]:
# Get list of copied files.
    output_files = [file for file in output_subfolder.rglob("*." + ext_in)]
    print(f"Found {len(output_files)} {ext_in} files in {output_subfolder.resolve()}")
    
    for output_file in output_files:
        print(f"Looking in {folder}")
        files = [file for file in folder.rglob("*." + "docx")]
        print(f"Found: {files}")
        unprocessed_files.extend(files)


for unprocessed_file in unprocessed_files:
    print(unprocessed_file)


In [52]:
input_files = unprocessed_files

for input_file in input_files:
    
    print(f"Opening {input_file}")
    # Open the input file as a Word document
    try :
        document = docx.Document(input_file)
    except BadZipFile:
        print(f"BadZipFile Error on opening {input_file}")
        continue
    
    # Get output filename.
    #output_file = input_file.with_suffix(".translated.docx").resolve()
    
    output_file = input_file.resolve()
    
    # Save the file.
    document.save(input_file)

    # From docx help
    #document = Document('existing-document-file.docx')
    #document.save('new-file-name.docx')
    
    # Translate the content
    paragraphs = translate_docx(output_file)
    
    print(f"Saved the translated file to {output_file}")

    #for paragraph in paragraphs:
    #    print(paragraph)


Opening drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/5 - Filling the Gaps.docx
Saved the translated file to /content/drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/5 - Filling the Gaps.docx
Opening drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/3 - Defining the Scenes.docx
Saved the translated file to /content/drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/3 - Defining the Scenes.docx
Opening drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/2 - Setting the Stage.docx
Saved the translated file to /content/drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/2 - Setting the Stage.docx
Opening drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/4 - Embodying the Text.docx
Saved the translated file to /content/drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/4 - Embodying the Text.docx
Opening drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/1 - Hear and Heart.docx
Saved the translated file to /content/drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/1 - Hear and Hea

In [37]:
output_folder = Path("drive/MyDrive/EIL-Mark-TokPisin")
ext = 'docx'
output_files = [file for file in output_folder.rglob("*.docx")]